# What's in an online dating profile? 

People say a lot about themselves in online dating profiles, especially on sites like OKCupid that encourage people to answer questions. Thus, we can learn a lot about people by studying what they write. OKC has made some of their profile data from San Fransisco public. We will be using that data in this lab to explore different cultural questions. 

Our first question is whether and how men and women talk about themselves differently in their profiles. Popular culture is constantly telling us that men and women have different interests, hobbies, and relationship goals. Yet there are also many examples of women who like stereotypically masculine things and men who like feminine ones. This is especially interesting in online dating, because people are seeking partners with similar interests and relationship goals. Finding a partner would be hard for straight men and women if these two groups had very different interests. 

OKC gave us 2,420,409 profiles though -- way too much to read! Computers can read them all and tell us how common different words are. So our first approach will be simple. We can ask 
1. Which words are used the most by men and women? 
2. Which words are used often by men but not women, and vice versa? 

At the end of the lab, you'll be able to ask this question about other social groups too (like sexual orientation, race/ethnicity, and level of education).

### Import the libraries we'll use.
`%matplotlib inline` lets us see charts and plots right here in the notebook!

In [ ]:
from collections import Counter

import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import regexp_tokenize 
from nltk.stem.snowball import SnowballStemmer
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt

%matplotlib inline

### Read our data.

In [ ]:
profiles = pd.read_csv('data/profiles.csv')

In [ ]:
#random subsample so this doesn't crash
profiles = profiles.sample(20000)
profiles = profiles.reset_index(drop=True)
profiles.columns

### A little housekeeping...
Expand for more.

- The OKC data has 10 different columns with profile text, one for each long-answer question in users' profiles.
- We want to look at all of the profile text, so this cell merges it all together in a new column called `text`.

### The code

In [ ]:
essay_cols = ['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 
              'essay7', 'essay8', 'essay9']

def concat(row, cols):
    tmp = []
    for c in cols:
        tmp.append(str(row[c]))
    new = '\n'.join(tmp)
    return new

profiles['text'] = profiles.apply(concat, axis=1, cols=essay_cols)

profiles = profiles[['age', 'body_type', 'diet', 'drinks', 'drugs', 'education', 
                     'ethnicity', 'height', 'income', 'job', 'last_online', 
                     'location', 'offspring', 'orientation', 'pets', 'religion', 
                     'sex', 'sign', 'smokes', 'speaks', 'status', 'text']]

profiles.head(3)

#### Let's peak at an example of the text so we know what we're working with.

In [ ]:
profiles.text[5]

## Tokenizing text

### We want to split the text into words.
Expand for details

- We can do this by applying the `split()` function to text in every profile. 
- Notice, however, that this is a little messy.
    - `split()` is just cutting up the text based on the spaces, leaving the punctuation and some HTML things mized in with our words.

### A first try

In [ ]:
tmp = profiles['text'].apply(lambda x: x.split())
tmp.head()

### Getting text from words
Expand for details

Here we define a function to clean up the text a bit more. It does a few things:
- Removes HTML code from the text using BeautifulSoup. (Remember, we want just the words people actually typed.) 
- Converts all of the text to lowercase, so that `Hello`, `hello`, `"HeLlO`, and `HELLO` all look the same to the computer.
- Uses the Natural Language Tool Kit (`nltk`) to tokenize the remaining text. 
    - "Tokenize" is jargon for splitting text into "tokens." Tokens are usually words, but they could be sentences, paragraphs, letters, or whatever we needed. 
    - The nltk tokenizers are much smarter than the simple `string.split()` function we used before. This one (which we imported in the beginning) selects the words, but ignores the whitespace and punctuation.

### A second try

#### Removing stop words

In [ ]:
keep_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 
              'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 
              'himself', 'she', 'her', 'hers', 'herself', 'they', 'them', 'their',
              'theirs', 'themselves']

sw = set(stopwords.words('english'))

for k in keep_words:
    sw.discard(k) #could use remove if we wanted keyerrors
    
print(sw)

In [ ]:
def clean(text):
    t = BeautifulSoup(text, 'lxml').get_text()
    
    bad_words = ['http', 'www', '\nnan']
    for b in bad_words:
        t = t.replace(b, '')
    
    t = t.lower()
    t = regexp_tokenize(t, '\w+')
    
    final = []
    for w in t:
        if w not in sw:
            final.append(w)
    
    return final

profiles['tokens'] = profiles['text'].apply(clean)
profiles.tokens.head()

## Comparing the words used by men and women

In [ ]:
men = profiles[(profiles['sex'] == 'm') & (profiles['orientation'] == 'straight')]
women = profiles[(profiles['sex'] == 'f') & (profiles['orientation'] == 'straight')]

men.tokens.head()

#### Counting how often each gender uses each word

In [ ]:
def flatten(series):
    l = []
    for x in series:
        l.extend(x) #each x is a list we want to unnest
    return l

tmp = flatten(men.tokens)

mens_words = Counter(tmp)
mens_words.most_common(10)

In [ ]:
tmp = flatten(women.tokens)

womens_words = Counter(tmp)
womens_words.most_common(10)

#### Convert those word counts to frequencies (percent of total words)

In [ ]:
tmp = {'women': womens_words,
       'men': mens_words
      }

popular_words = pd.DataFrame(tmp)

popular_words['men'] = (popular_words['men'] /  popular_words['men'].sum())*100
popular_words['women'] = (popular_words['women'] /  popular_words['women'].sum())*100

popular_words.sort_values(by='men', inplace=True, ascending=False)
popular_words.head().round(2)

In [ ]:
popular_words['max'] = popular_words.max(axis=1)
popular_words = popular_words.sort_values(by='max', ascending=False)
popular_words.head(10).round(2)

#### See the distribution of word popularity

In [ ]:
popular_words['max'].hist(bins=100)

#### Look at just the 1000 most popular words

In [ ]:
popular_words = popular_words.head(1000)
print(popular_words.shape)
popular_words['max'].hist(bins=100)

### Figure out which words are popular with one gender but not the other

In [ ]:
def times_diff(row):
    if row.men > row.women:
        return row.men / row.women
    else:
        return -1 * (row.women / row.men)
    
popular_words['times_diff'] = popular_words.apply(times_diff, axis=1)
popular_words = popular_words.sort_values(by='max', ascending=False)

print('Most popular words:')
popular_words.head(10).round(3)

In [ ]:
popular_words = popular_words.sort_values(by='times_diff', ascending=False)

print('Words men use more than women:')
popular_words.head(15).round(3)

In [ ]:
popular_words = popular_words.sort_values(by='times_diff', ascending=True)

print('Words women use more than men:')
popular_words.head(15).round(3)

### Stemming

In [ ]:
#snowball English (aka porter2) is improved over the porter stemmer
stemmer = SnowballStemmer("english") 

def stem(t):
    out = []
    for w in t:
        out.append(stemmer.stem(w))
    return out

profiles['stems'] = profiles['tokens'].apply(stem)
profiles.stems.head()

#### Let's try it again with stems this time

In [ ]:
# functions for summarizing word use by a trait
def times_diff2(row, group, ref):
    if row[ref] > row[group]:
        return -1 * (row[ref] / row[group])
    else:
        return row[group] / row[ref]

#normally we wouldn't paste this function here but it helps to show in the lab
def flatten(series):
    l = []
    for x in series:
        l.extend(x) #each x is a list we want to unnest
    return l
    
def flatten2(series):
    l = []
    for x in series:
        tmp = set(x) #make the tokens into a set, thus dropping repeats
        tmp = list(tmp) #turn it back into a list we can attach to the other lists
        l.extend(tmp) 
    return l

def word_use(df, att, ref, per_person=False):
    types = list(df[att].value_counts().index.values)
    data = {}
    lens = {}
    
    for t in types:
        tmp = df[df[att] == t].stems
        lens[t] = len(tmp)
        
        if per_person:
            tmp = flatten2(tmp)
        else:
            tmp = flatten(tmp)
        data[t] = Counter(tmp)
        
    popular_words = pd.DataFrame(data)
    
    for t in types:
        n = 0
        
        if per_person:
            n = lens[t]
        else:
            n = popular_words[t].sum()
        
        popular_words[t] = (popular_words[t] / n) * 100
    
    #find overall most popular words, select the top 1000
    popular_words['max'] = popular_words.max(axis=1)
    popular_words = popular_words.sort_values(by='max', ascending=False)
    popular_words = popular_words.head(1000)
    
    for t in types:
        if t != ref:
            popular_words['times_diff_'+t] = popular_words.apply(times_diff2, 
                                                                 group=t, ref=ref, 
                                                                 axis=1)

    
    return popular_words

In [ ]:
popular_words = word_use(profiles, att='sex', ref='m')
popular_words = popular_words.sort_values(by='times_diff_f', ascending=True)
print("Men's words:")
popular_words.head(10).round(2)

In [ ]:
popular_words = popular_words.sort_values(by='times_diff_f', ascending=False)
print("Women's words:")
popular_words.head(10).round(2)

#### Let's try it again, but with percent of profiles rather than percent of words
not all profiles have the same number of words

In [ ]:
popular_words = word_use(profiles, att='sex', ref='m', per_person=True)
popular_words = popular_words.sort_values(by='times_diff_f', ascending=True)
print("Men's words:")
popular_words.head(10).round(2)

In [ ]:
popular_words = popular_words.sort_values(by='times_diff_f', ascending=False)
print("Women's words:")
popular_words.head(10).round(2)

## Try it with another trait

In [ ]:
result = word_use(profiles, att='orientation', ref='straight', per_person=True)

In [ ]:
result = result.sort_values(by='times_diff_gay', ascending=False)
result.head(10).round(2)

In [ ]:
result = result.sort_values(by='times_diff_bisexual', ascending=False)
result.head(10).round(2)

In [ ]:
result = result.sort_values(by='times_diff_gay', ascending=True)
result.head(10).round(2)

In [ ]:
profiles.ethnicity.value_counts().head(20)